# Accessing NASA IMERG Final Run V07 Precipitation Data (May 2024)

**Author:** Gaurav Somani  
**Course:** Earth System Data Processing 1 (WS 2025/26)  
**Notebook:** load_nasa_imerg.ipynb

This notebook demonstrates programmatic access and download of NASA IMERG Final Run (V07B) precipitation data for a 5-day period in May 2024. It includes authentication setup, API download procedures, timing, file inspection, and a basic visualization.


In [28]:
import requests
import os
from pathlib import Path
from datetime import datetime, timedelta
import h5py
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [29]:
# Create an authenticated session (uses ~/.netrc)
session = requests.Session()
session.trust_env = True   # VERY IMPORTANT: tells requests to use .netrc

# Quick test request (small, fast)
test_url = "https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/"

r = session.get(test_url)
print("Status:", r.status_code)

Status: 200


In [30]:
# IMERG half-hourly folder (one full day = 48 files)
base_url = "https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.07/2025/122/"

print("Dataset directory:", base_url)


Dataset directory: https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGHH.07/2025/122/


In [31]:
# Fetch the directory listing
html = session.get(base_url).text
soup = BeautifulSoup(html, "html.parser")

# Extract all HDF5 filenames
files = [a.text for a in soup.find_all("a") if a.text.endswith(".HDF5")]

len(files), files[:3]   # show count + first 3


(48,
 ['3B-HHR.MS.MRG.3IMERG.20250502-S000000-E002959.0000.V07B.HDF5',
  '3B-HHR.MS.MRG.3IMERG.20250502-S003000-E005959.0030.V07B.HDF5',
  '3B-HHR.MS.MRG.3IMERG.20250502-S010000-E012959.0060.V07B.HDF5'])

In [32]:
# Make a folder to store the day's files
Path("imerg_day").mkdir(exist_ok=True)

for i, fn in enumerate(files, start=1):
    url = base_url + fn
    out_path = Path("imerg_day") / fn

    # Skip if already downloaded
    if out_path.exists():
        print(f"[{i}/{len(files)}] Already downloaded:", fn)
        continue
    
    # Download file
    r = session.get(url, stream=True)
    if r.status_code == 200:
        with open(out_path, "wb") as f:
            for chunk in r.iter_content(8192):
                f.write(chunk)
        print(f"[{i}/{len(files)}] Downloaded:", fn)
    else:
        print(f"[{i}/{len(files)}] FAILED:", fn)


[1/48] Downloaded: 3B-HHR.MS.MRG.3IMERG.20250502-S000000-E002959.0000.V07B.HDF5
[2/48] Downloaded: 3B-HHR.MS.MRG.3IMERG.20250502-S003000-E005959.0030.V07B.HDF5
[3/48] Downloaded: 3B-HHR.MS.MRG.3IMERG.20250502-S010000-E012959.0060.V07B.HDF5
[4/48] Downloaded: 3B-HHR.MS.MRG.3IMERG.20250502-S013000-E015959.0090.V07B.HDF5
[5/48] Downloaded: 3B-HHR.MS.MRG.3IMERG.20250502-S020000-E022959.0120.V07B.HDF5
[6/48] Downloaded: 3B-HHR.MS.MRG.3IMERG.20250502-S023000-E025959.0150.V07B.HDF5
[7/48] Downloaded: 3B-HHR.MS.MRG.3IMERG.20250502-S030000-E032959.0180.V07B.HDF5
[8/48] Downloaded: 3B-HHR.MS.MRG.3IMERG.20250502-S033000-E035959.0210.V07B.HDF5
[9/48] Downloaded: 3B-HHR.MS.MRG.3IMERG.20250502-S040000-E042959.0240.V07B.HDF5
[10/48] Downloaded: 3B-HHR.MS.MRG.3IMERG.20250502-S043000-E045959.0270.V07B.HDF5
[11/48] Downloaded: 3B-HHR.MS.MRG.3IMERG.20250502-S050000-E052959.0300.V07B.HDF5
[12/48] Downloaded: 3B-HHR.MS.MRG.3IMERG.20250502-S053000-E055959.0330.V07B.HDF5
[13/48] Downloaded: 3B-HHR.MS.MRG.3IM

In [33]:
sample_file = "imerg_day/" + files[0]
print("Using:", sample_file)

f = h5py.File(sample_file, "r")

print("Top-level keys:", list(f.keys()))
print("Grid keys:", list(f["Grid"].keys()))


Using: imerg_day/3B-HHR.MS.MRG.3IMERG.20250502-S000000-E002959.0000.V07B.HDF5
Top-level keys: ['Grid']
Grid keys: ['Intermediate', 'nv', 'lonv', 'latv', 'time', 'lon', 'lat', 'time_bnds', 'lon_bnds', 'lat_bnds', 'precipitation', 'randomError', 'probabilityLiquidPrecipitation', 'precipitationQualityIndex']


In [35]:
def print_hdf5_structure(g, indent=0):
    for key in g.keys():
        item = g[key]
        print("  " * indent + "📁", key)
        if isinstance(item, h5py.Group):
            print_hdf5_structure(item, indent + 1)
        else:
            print("  " * (indent+1) + f"shape = {item.shape}, dtype = {item.dtype}")

# Use it:
print_hdf5_structure(f)


📁 Grid
  📁 Intermediate
    📁 MWprecipitation
      shape = (1, 3600, 1800), dtype = float32
    📁 MWprecipSource
      shape = (1, 3600, 1800), dtype = int16
    📁 MWobservationTime
      shape = (1, 3600, 1800), dtype = int16
    📁 IRprecipitation
      shape = (1, 3600, 1800), dtype = float32
    📁 IRinfluence
      shape = (1, 3600, 1800), dtype = int16
    📁 precipitationUncal
      shape = (1, 3600, 1800), dtype = float32
  📁 nv
    shape = (2,), dtype = int32
  📁 lonv
    shape = (2,), dtype = int32
  📁 latv
    shape = (2,), dtype = int32
  📁 time
    shape = (1,), dtype = int32
  📁 lon
    shape = (3600,), dtype = float32
  📁 lat
    shape = (1800,), dtype = float32
  📁 time_bnds
    shape = (1, 2), dtype = int32
  📁 lon_bnds
    shape = (3600, 2), dtype = float32
  📁 lat_bnds
    shape = (1800, 2), dtype = float32
  📁 precipitation
    shape = (1, 3600, 1800), dtype = float32
  📁 randomError
    shape = (1, 3600, 1800), dtype = float32
  📁 probabilityLiquidPrecipitation
    s

In [36]:
def explore_hdf5(name, obj):
    print("🔹", name)
    for k, v in obj.attrs.items():
        print("    ▫ attribute:", k, "=", v)

f.visititems(explore_hdf5)


🔹 Grid
    ▫ attribute: GridHeader = b'BinMethod=ARITHMETIC_MEAN;\nRegistration=CENTER;\nLatitudeResolution=0.1;\nLongitudeResolution=0.1;\nNorthBoundingCoordinate=90;\nSouthBoundingCoordinate=-90;\nEastBoundingCoordinate=180;\nWestBoundingCoordinate=-180;\nOrigin=SOUTHWEST;\n'
🔹 Grid/Intermediate
🔹 Grid/Intermediate/IRinfluence
    ▫ attribute: DimensionNames = b'time,lon,lat'
    ▫ attribute: coordinates = b'time lon lat'
    ▫ attribute: _FillValue = -9999
    ▫ attribute: CodeMissingValue = b'-9999'
    ▫ attribute: LongName = b'\nRelative influence of infrared precipitation in the merged\n                        microwave-infrared precipitation estimate\n'
    ▫ attribute: DIMENSION_LIST = [array([<HDF5 object reference>], dtype=object)
 array([<HDF5 object reference>], dtype=object)
 array([<HDF5 object reference>], dtype=object)]
🔹 Grid/Intermediate/IRprecipitation
    ▫ attribute: DimensionNames = b'time,lon,lat'
    ▫ attribute: Units = b'mm/hr'
    ▫ attribute: units = b'mm/h